## Core Functions
- Report the range of times for each event for the given data set (done)
- Report the median time for each for the selected data set (done)
- Report fastest for each event along with that person and the rest of their time (EX "Christina Hepper has the fastest Swim at _____ and had the following other times...., Jamie Fend had the fastest T1 at _____ and had teh following other times...." (done, but need to improve formatting of result)
- Create functions
-- function to convert to integer minutes (done)
-- function for performance standardizing (done)
- Compute the fastest theoretical time (done)
- Compute the median theoretical time
- Report how selected Bib Number did relative to lead (EX "989 was 0.9x at swimming, 1.1x at T1....")
- Create a line plot with each event on the X, the Standardized Time on the Y (standardize to fastest?) and each line being an athlete, highlight the user and the median line
- Need to cumsum the colums

## Future
- Use mongo db to store data
- Automatically scrape the results
- Compute a "competitiveness" score for each race (like FSAE world: https://fs-world.org/)
- Calculate the percentile for eahc event for the user

In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as goS
from sqlalchemy import create_engine
pd.set_option("precision", 2)
import time

In [4]:
#data = pd.read_excel('santa_cruz_data.xlsx', header = 0, index_col=None)
data = pd.read_csv('Santa-Cruz-Sprint.csv')
data.head()


,Unnamed: 0,Bib,Name,Team Name,City,Start Time,Swim,T1,Bike,T2,Run,Chip Elapsed,Division,Div Place,Age,Age Place,Gender,Gender Place
0,0,1051,MANUEL HUECKEL,TEAM WY55,SAN FRANCISCO,09:35:00,00:10:11,00:03:08,00:31:19,00:00:44,00:19:10,01:04:34,SPRINT-AGE GROUP,1,23,O/A,M,1
1,1,978,TYLER CURRAN,NaN,WHITE PLAINS,09:35:00,00:09:28,00:02:59,00:31:42,00:00:52,00:20:14,01:05:16,SPRINT-AGE GROUP,2,24,O/A,M,2
2,2,924,BRIAN BERGER,GOLD COUNTRY TRIATHLON CLUB,LINCOLN,09:35:00,00:09:33,00:03:53,00:33:12,00:01:25,00:22:02,01:10:08,SPRINT-AGE GROUP,3,25,O/A,M,3
3,3,1187,CLAY ROBBINS,NaN,REDWOOD CITY,09:35:00,00:11:30,00:03:42,00:34:03,00:01:37,00:19:26,01:10:19,SPRINT-AGE GROUP,4,30,1,M,4
4,4,943,ANDREW BUNNER,NaN,BELMONT,09:40:00,00:11:46,00:03:49,00:33:34,00:01:06,00:20:54,01:11:12,SPRINT-AGE GROUP,5,44,1,M,5


We need to convert the DataTime objects contatined in the data frame to integer minutes to do all of our nice math. Use the function and create new columsn to hold the data.

Create a second set of parameters for cumulative times to use later

In [24]:
test_df = pd.to_datetime(data["Start Time"])
#print(type(data.iloc[2]["Swim"]))
playtime = data.iloc[2]["Swim"]
#playtime

#testhing = time.strftime(playtime, )

def convertTime (time):
    temp = time.split(':')
    timeMinutes = (int(temp[0])*60)+int(temp[1])+int(temp[2])/60
    return timeMinutes

print(convertTime(playtime))

9.55


In [25]:
#create a function to convert the time to integer seconds
#To create a new column, use the [] brackets with the new column name at the left side of the assignment.

def convertTime (time):
    temp = time.split(':')
    timeMinutes = (int(temp[0])*60)+int(temp[1])+int(temp[2])/60
    return timeMinutes

#convert to integers
data["Swim Minutes"] = data["Swim"].apply(convertTime)
data["T1 Minutes"] = data["T1"].apply(convertTime)
data["Bike Minutes"] = data["Bike"].apply(convertTime)
data["T2 Minutes"] = data["T2"].apply(convertTime)
data["Run Minutes"] = data["Run"].apply(convertTime)
#data["Elapsed Minutes"] = data["Chip Elapsed"].apply(convertTime)

#create cumulative times
data["Swim+T1"]=data["Swim Minutes"]+data["T1 Minutes"]
data["Plus Bike"]=data["Swim+T1"]+data["Bike Minutes"]
data["Plus T2"]=data["Plus Bike"]+data["T2 Minutes"]
data["Total"]=data["Plus T2"]+data["Run Minutes"]

Get statistical data about the event.

In [26]:
summary = data.describe()
data[["Swim Minutes", "T1 Minutes", "Bike Minutes", "T2 Minutes", "Run Minutes"]].describe()

,Swim Minutes,T1 Minutes,Bike Minutes,T2 Minutes,Run Minutes
count,369.00,369.00,369.00,369.00,369.00
mean,14.01,6.06,35.91,1.97,25.52
std,7.70,3.95,18.46,2.97,16.02
min,0.00,0.00,0.00,0.00,0.00
25%,12.48,4.35,35.98,1.08,22.82
50%,15.18,6.22,42.03,1.68,27.28
75%,18.38,8.17,46.87,2.38,32.22
max,36.28,27.78,68.60,43.70,131.02


In [29]:
#assign user variable
user = "SAMANTHA DEWEES"
index = data.index[data['Name']==user].tolist()
data.loc[index, ["Name", "Age", "Gender Place", "Swim", "T1", "Bike", "T2", "Run"]]

,Name,Age,Gender Place,Swim,T1,Bike,T2,Run
12,SAMANTHA DEWEES,25.0,13.0,00:15:02,00:07:04,00:39:39,00:02:13,00:23:38


In [30]:
def getMinTime (stage):
    minTime = data[stage].min()
    minIndex = data[stage].idxmin()
    #print("minTime is", minTime)
    #print("minIndex is", minIndex)
    return minTime, minIndex
    
def printResults (minIndex):
    #need to make this so it doesnt print reduntant colums or prints something nicely formatted
    print(data.iloc[minIndex,:])

In [31]:
events = ['Swim Minutes', 'T1 Minutes', 'Bike Minutes', 'T2 Minutes', 'Run Minutes']

shortestTime = 0
for stage in events:
    minTime, minIndex = getMinTime(stage)
    print("the fastest for ", stage)
    printResults(minIndex)
    shortestTime = shortestTime + minTime

print("the shortest possible time is: ", shortestTime)
#need to work on formatting

the fastest for  Swim Minutes
Bib                         1288
Name            CAROLE YOSHIWARA
Team Name                    NaN
City                   SUNNYVALE
Start Time              09:45:00
Swim                    00:11:51
T1                      00:05:18
Bike                    00:42:36
T2                      00:01:27
Run                     00:24:47
Chip Elapsed            01:26:02
Division        SPRINT-AGE GROUP
Div Place                   52.0
Age                         33.0
Age Place                      2
Gender                         F
Gender Place                10.0
Swim Minutes               11.85
T1 Minutes                   5.3
Bike Minutes                42.6
T2 Minutes                  1.45
Run Minutes                24.78
Swim+T1                    17.15
Plus Bike                  59.75
Plus T2                     61.2
Total                      85.98
Name: 9, dtype: object
the fastest for  T1 Minutes
Bib                                         1006
Name        

In [32]:
reduced2 = data[["Name","Swim Minutes","Swim+T1","Plus Bike","Plus T2","Total","Gender Place"]]
reduced2["Start"] = 0
reduced2 = reduced2.drop(25)
reduced2

# color_nums = reduced2['Gender Place'].tolist()
# color_names = reduced2['Name'].tolist()
# colors_zip = zip(color_nums, color_names)

# colors_zip

<ipython-input-32-16d8c57e1e0f>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Name,Swim Minutes,Swim+T1,Plus Bike,Plus T2,Total,Gender Place,Start
0,CHRISTINA HEPPER,14.00,18.25,54.28,55.62,79.28,1.0,0
1,JAMIE FEND,15.23,19.47,57.60,58.93,81.73,2.0,0
2,ALEXANDRA LUZI,12.83,19.52,57.58,59.38,82.12,3.0,0
3,SUMI HACHMANN,14.27,18.93,57.53,58.72,82.65,4.0,0
4,DENISE BUTLER,12.57,17.18,57.32,58.13,82.90,5.0,0
5,SHARON BARCLAY,14.52,18.95,55.17,56.55,83.22,6.0,0
6,AMY PEARSON,13.50,17.98,56.98,58.67,83.47,7.0,0
7,NOEL KOSIEK,13.57,17.92,56.97,58.18,85.33,8.0,0
8,CATHERINE LONDON,14.13,19.78,59.65,62.62,85.88,9.0,0
9,CAROLE YOSHIWARA,11.85,17.15,59.75,61.20,85.98,10.0,0


In [33]:
dimensions = list([
            dict(range = [0, 1],
                label = 'Start', values = reduced2['Start']),            
            dict(range = [reduced2["Swim Minutes"].min(), reduced2["Swim Minutes"].max()],
                label = 'Time After Swim', values = reduced2['Swim Minutes']),
            dict(range = [reduced2["Swim+T1"].min(), reduced2["Swim+T1"].max()],
                label = 'Time After First Transition', values = reduced2['Swim+T1']),
            dict(range = [reduced2["Plus Bike"].min(), reduced2["Plus Bike"].max()],
                label = 'Time After Bike', values = reduced2['Plus Bike']),
            dict(range = [reduced2["Plus T2"].min(), reduced2["Plus T2"].max()],
                label = 'Time After Second Transition', values = reduced2['Plus T2']),
            dict(range = [reduced2["Total"].min(), reduced2["Total"].max()],
                label = 'Total Time', values = reduced2['Total']),
            dict(range=[0,reduced2['Gender Place'].max()], tickvals = reduced2['Gender Place'], ticktext = reduced2['Name'],
                       label='Competitor', values=reduced2['Gender Place'])
        ])

fig = go.Figure(data=go.Parcoords(line = dict(color = reduced2['Gender Place'],
                   colorscale = [[.0,'rgba(255,0,0,0.1)'],[0.2,'rgba(0,255,0,0.1)'],[.4,'rgba(0,0,255,0.1)'], 
                                 [.6,'rgba(0,255,255,0.1)'], [.8, 'rgba(255,0,255,0.1)'], [1, 'rgba(0,0,0,0.1)']]), dimensions=dimensions))

fig.update_layout(
    title="Triathalon Results",
    width=1920,
    height=1080
)

fig.show()

In [15]:
engine = create_engine('sqlite:///data.db', echo=False)

In [16]:
sql_df = pd.read_sql('data.db', con=engine, index_col='ID')

In [17]:
sql_df.head()

,Bib,Name,Team Name,City,Start Time,Swim,T1,Bike,T2,Run,Chip Elapsed,Division,Div Place,Age,Age Place,Gender,Gender Place
ID,,,,,,,,,,,,,,,,,
0,1051,MANUEL HUECKEL,TEAM WY55,SAN FRANCISCO,09:35:00,00:10:11,00:03:08,00:31:19,00:00:44,00:19:10,01:04:34,SPRINT-AGE GROUP,1,23,O/A,M,1
1,978,TYLER CURRAN,,WHITE PLAINS,09:35:00,00:09:28,00:02:59,00:31:42,00:00:52,00:20:14,01:05:16,SPRINT-AGE GROUP,2,24,O/A,M,2
2,924,BRIAN BERGER,GOLD COUNTRY TRIATHLON CLUB,LINCOLN,09:35:00,00:09:33,00:03:53,00:33:12,00:01:25,00:22:02,01:10:08,SPRINT-AGE GROUP,3,25,O/A,M,3
3,1187,CLAY ROBBINS,,REDWOOD CITY,09:35:00,00:11:30,00:03:42,00:34:03,00:01:37,00:19:26,01:10:19,SPRINT-AGE GROUP,4,30,1,M,4
4,943,ANDREW BUNNER,,BELMONT,09:40:00,00:11:46,00:03:49,00:33:34,00:01:06,00:20:54,01:11:12,SPRINT-AGE GROUP,5,44,1,M,5
